* 펀드슈퍼마켓 서비스 내 공개포트폴리오에 있는 포트폴리오 정보를 수집하여 시기별로 사람들이 관심이 있는 펀드는 무엇인지 살펴보자
* 참고 : http://fundsupermarket.co.kr/fmm/FMM1010501/main.do
* 하나의 공개 포트폴리오는 2~5개의 펀드로 구성되어 있다.
* 수집할 데이터의 양식은 다음과 같이 하자
* 포트폴리오제목 / 펀드 1 / 펀드 2 / 펀드 3 / 펀드 4 / 펀드 5 / 등록일자

In [1]:
# 크롤링 및 전처리에 필요한 패키지 로드

library(rvest)
library(httr)
library(dplyr)
library(stringr)

Loading required package: xml2

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
# 크롤링할 페이지의 url을 변수로 지정하여 for문을 통해 페이지 변경을 진행
url = "http://www.fundsupermarket.co.kr/fmm/FMM1010501/main.do?iPageNum="

# 크롤링할 변수를 기록할 벡터를 생성
title_record <- NULL
fund_record <- NULL
day_record <- NULL
fund_len_record <- NULL

In [ ]:
# 데이터 수집

for(i in 1:597){
  tmp_url <- paste0(url,i)
  data <- GET(tmp_url)
  read_file <- read_html(data)
  
  # 포트폴리오명 크롤링
  title <- read_file %>%
    html_nodes(css = '.cartp-tit-area') %>%
    html_nodes('a') %>%
    html_text() %>%
    unlist()
  title <- title[((1:length(title)) %% 5) == 1]
  title_record <- append(title_record, title)
  
  # 포트폴리오 내 펀드 크롤링
  portfolio <- read_file %>%
    html_nodes(css = '.tbl-vtype1')
  portfolio_fund <- portfolio[-1] %>%
    html_nodes('.txt-hidden') %>%
    html_text()
  num_portfolio <- length(portfolio)-1
  nums_portfolio <- NULL
  for(j in 2:(num_portfolio+1)){
    nums_portfolio[j-1] <- gregexpr('fund_code', portfolio[j]) %>%
      unlist() %>%
      length()
  }
  fund_record <- append(fund_record, portfolio_fund)
  fund_len_record <- append(fund_len_record, nums_portfolio)
  
  # 등록일자 크롤링
  day <- NULL
  path <- paste0("//*[@id=\"container\"]/div[1]/div[3]/div[", c(1,3,5,7,9) ,"]/div[2]/span[8]")
  for(k in 1:num_portfolio){
    day[k] <- portfolio[k+1] %>%
      html_nodes(xpath=path[k]) %>%
      html_text()
  }
  day <- gsub("\n|\t| ","",day)
  day <- str_sub(day, 1, 10)
  day_record <- append(day_record, day)
  
  cat('\n',i)
}

In [ ]:
# 데이터를 저장할 데이터프레임을 생성
df <- data.frame(matrix(nrow=length(title_record), ncol=7))
colnames(df) <- c("title", 'fund1', 'fund2', 'fund3', 'fund4', 'fund5', 'date')
tmp <- 0

# 기존에 수집한 데이터를 양식에 맞게 데이터프레임에 저장
for(i in 1:length(title_record)){
  df[i,1] <- title_record[i]
  for(j in 1:fund_len_record[i]){
    df[i,(j+1)] <- fund_record[tmp+j]
  }
  tmp <- tmp + fund_len_record[i]
  df[i,7] <- day_record[i]
  cat('\n',i)
}

write.csv(df, "../data/portfolio.csv")

* 분석은 다시 진행하자